In [58]:
%matplotlib inline
import math
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns; sns.set(style='ticks', color_codes=True)

# from pandas.plotting import parallel_coordinates
from sklearn import preprocessing
from operator import itemgetter

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import GaussianNB
from sklearn.externals import joblib

# pd.set_option('display.max_columns', 150)
pd.options.display.max_seq_items = 200

In [59]:
TRAIN_DATA = 'data/train.csv'
TEST_DATA = 'data/test.csv'
CLEAN_TRAIN_DATA = 'data/clean_train_data.csv'
NOISE_COLUMNS = ['Key', 'No', 'District', 'City', 'Hour', 'Minute', 'Date', 'Month', 'Year',
                'JunctionControl', 'dayInWeek', 'IsDayOrNight', 'CollisionDirection', 'DetailCause',
                'Tricycle', 'Weather']
NORMALIZE_COLUMNS = ['DriversKilled', 'DriversInjured', 'PassengerInjured', 'PassengerKilled', 'PedestrianKilled',
                 'PedestrianInjured', 'NumPedestrianVictim']
ONE_HOT_COLUMNS = ['CollisionType', 'JunctionType']

In [60]:
#Normalization

def normalize_df(df_base, columns):
    df = df_base.copy()
    for column in columns:
        df[column] = normalize_max_unknown(df[column])
    
    return df

def normalize_max_unknown(vector):
    min = np.min(vector)
    max = np.max(vector)
        
    return [(x - float(min)) / (float(max) - float(min)) for x in vector]

In [61]:
#Data Visualization

def parallel_lines(df, target_category):
    plt.pyplot.figure(figsize=(30,10))

    parallel_plt = parallel_coordinates(df, target_category)
#     plt.pyplot.savefig('%s_parallel.png' % target_category)
    
def scatter_plot(df, target_category):
    scatter_plt = sns.pairplot(df, hue=target_category)
    scatter_plt.savefig('%s_scatter.png' % target_category)

In [62]:
def generate_arr(dataset, classification):
    classification_arr = dataset[classification].values
    del dataset[classification]
    dataset_arr = dataset.values

    return dataset_arr, classification_arr

def impute_columns(df_base, columns):
    df = df_base.copy()
    for column in columns:
        df[column] = df[column].fillna(float(0.00))
        
    return df

In [63]:
mmda_df = pd.read_csv(TRAIN_DATA)
mmda_df = mmda_df.drop(NOISE_COLUMNS, axis=1)

In [64]:
mmda_df_one_hot = pd.get_dummies(mmda_df, prefix=ONE_HOT_COLUMNS, columns=ONE_HOT_COLUMNS)

imputer = Imputer(strategy='most_frequent')
imputer.fit(mmda_df_one_hot)
imputer_result = imputer.transform(mmda_df_one_hot)

mmda_df_clean = pd.DataFrame(imputer_result, columns=mmda_df_one_hot.columns)
# mmda_df_clean = impute_columns(mmda_df_one_hot, NORMALIZE_COLUMNS)
mmda_df_clean = normalize_df(mmda_df_clean, NORMALIZE_COLUMNS)

mmda_df_clean.to_csv(CLEAN_TRAIN_DATA)

In [65]:
# mmda_df_clean = pd.read_csv(CLEAN_TRAIN_DATA)
# corr_matrix = mmda_df_clean.corr()
# corr_matrix['Classification_1'].sort_values(ascending=False)

In [66]:
mmda_arr, classification_arr = generate_arr(mmda_df_clean, 'Classification')
# labels = [[x] for x in classification_arr]
# binary_labels = MultiLabelBinarizer().fit_transform(labels)
# binary_labels

In [85]:
mmda_df_clean
len(mmda_arr[0])

123

In [68]:
clf = GaussianNB()
clf.fit(mmda_arr, classification_arr)
joblib.dump(clf, 'mmda_predictor.pkl')

['mmda_predictor.pkl']

In [86]:
#Model Validation

NOISE_TEST_COLUMNS = ['Key', 'District', 'City', 'Hour', 'Minute', 'Date', 'Month', 'Year',
                'JunctionControl', 'dayInWeek', 'IsDayOrNight', 'CollisionDirection', 'DetailCause',
                'Tricycle', 'Weather']

mmda_df_test = pd.read_csv(TEST_DATA)
mmda_df_test = mmda_df_test.drop(NOISE_TEST_COLUMNS, axis=1)

mmda_df_one_hot = pd.get_dummies(mmda_df_test, prefix=ONE_HOT_COLUMNS, columns=ONE_HOT_COLUMNS)

# mmda_df_one_hot['DriversKilled'].iloc[0] = float(0)
# mmda_df_one_hot['PassengerKilled'].iloc[0] = float(0)
# mmda_df_one_hot['PedestrianKilled'].iloc[0] = float(0)

imputer = Imputer(strategy='most_frequent')
imputer.fit(mmda_df_one_hot)
imputer_result = imputer.transform(mmda_df_one_hot)

mmda_df_clean = pd.DataFrame(imputer_result, columns=mmda_df_one_hot.columns)
# mmda_df_clean = impute_columns(mmda_df_one_hot, NORMALIZE_COLUMNS)
mmda_df_clean = normalize_df(mmda_df_clean, NORMALIZE_COLUMNS)

mmda_df_test_clean['DriversKilled'] = mmda_df_test_clean['DriversKilled'].fillna(0.0)
mmda_df_test_clean['PassengerKilled'] = mmda_df_test_clean['PassengerKilled'].fillna(0.0)
mmda_df_test_clean['PedestrianKilled'] = mmda_df_test_clean['PedestrianKilled'].fillna(0.0)

mmda_test_arr = mmda_df_test_clean.values
len(mmda_test_arr[0])

C:\Users\anton\Anaconda2\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: invalid value encountered in double_scalars


114

In [84]:
pred = clf.predict(mmda_test_arr)
pred

ValueError: operands could not be broadcast together with shapes (16485,114) (123,) 